# Деревья решений

## Построение дерева

Опишем жадный алгоритм построения бинарного дерева решений:
1. Начинаем со всей обучающей выборки $X$, которую помещаем в корень $R_1$. 
2. Задаём функционал качества $Q(X, j, t)$ и критерий остановки. 
3. Запускаем построение из корня: $SplitNode(1, R_1)$

Функция $SplitNode(m, R_m)$
1. Если выполнен критерий остановки, то выход.
2. Находим наилучший с точки зрения $Q$ предикат: $j, t$: $[x_j<t]$
3. Помещаем предикат в вкршину и получаем с его помощью разбиение $X$ на две части: $R_{left} = \lbrace x|x_j<t \rbrace$ и $R_{right} = \lbrace x|x_j \geqslant t \rbrace$
4. Поместим $R_{left}$ и $R_{right}$ соответсвенно в левое и правое поддерево.
5. Рекурсивно повторяем $SplitNode(left, R_{left})$ и $SplitNode(right, R_{right})$.

В конце поставим в соответствие каждому листу ответ. Для задачи классификации - это самый частый среди объектов класс или вектор с долями классов (можно интерпретировать как вероятности):
$$ c_v = \arg \max_{k\in Y} \sum_{(x_i,y_i) \in R_v} [y_i=k]  $$

## Функционал качества для деревьев решений


Энтропия Шеннона для системы с N возможными состояниями определяется по формуле:
$$H = - \sum_{i=0}^{N} p_i\log_2p_i $$

где $p_i$ – вероятности нахождения системы в $i$-ом состоянии. 

Это очень важное понятие теории информации, которое позволяет оценить количество информации (степень хаоса в системе). Чем выше энтропия, тем менее упорядочена система и наоборот. С помощью энтропии мы формализуем функционал качества для разделение выборки (для задачи классификации).


### Задание 4.1

Реализуйте алгоритм построения дерева. Должны быть отдельные функции (методы) для расчёта энтропии (уже есть), для разделения дерева (используйте `pandas`), для подсчёта функционала качества $IG$, для выбора наилучшего разделения (с учетом признакоd и порогов), для проверки критерия остановки.

Для набора данных `iris` реализуйте алгоритм и минимум три из разными критерия остановки из перечисленных ниже:
* максимальной глубины дерева = 5
* минимального числа объектов в листе = 5
* максимальное количество листьев в дереве = 5
* purity (остановка, если все объекты в листе относятся к одному классу)

Реализуйте функцию `predict` (на вход функции подаётся датафрейм с объектами)

Оцените точность каждой модели с помощью метрики точность (`from sklearn.metrics import accuracy_score` или реализовать свою).

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import random
from pprint import pprint

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()

In [ ]:
class DecisionTree:
  def __init__(self):
    pass
  
  def fit(self, df):
    self.train = df
    self.result = pd.DataFrame([])

  def entropy(self,y):
    _, counts = np.unique(y, return_counts=True)
    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
    return entropy

  def compute_information_gain(self, df, left, right):
    if left.shape[0] == 0:
      return 0
    if right.shape[0] == 0:
      return 0
    HRv = self.entropy(df['y'])
    cntRv = df.shape[0]
    cntRleft = left.shape[0]
    cntRright = right.shape[0]
    HRleft = self.entropy(left['y'])
    HRright = self.entropy(right['y'])
    return HRv - ((cntRleft/cntRv)*HRleft + (cntRright/cntRv)*HRright)

  def find_best_split(self, df):
    grid = np.linspace(df[0].min(), df[0].max(), 10)
    best_t = grid[0]
    best_IG = -1
    for attr in range(df.shape[1]-1):
      grid = np.linspace(df[attr].min(), df[attr].max(), 10)
      for split_point in grid:
        left = df[df[attr] < split_point]
        right = df[df[attr] >= split_point]
        IG = self.compute_information_gain(df, left, right)
        if IG > best_IG:
          best_t, best_IG  = split_point, IG
          best_attr = attr
    return best_t, best_attr   
  
  def build_tree(self, df_train, df_test, criterion, depth, CNT):
    CNT += 2
    if self.stop_criterion(criterion, df_train, depth, CNT):
      if df_test.shape[0] > 0:
        df_test['y'] = df_train['y'].value_counts().idxmax()
        self.result = self.result.append(df_test)
        return self
    else:
      split_point, attr = self.find_best_split(df_train)
      self.build_tree(df_train[df_train[attr] < split_point], df_test[df_test[attr] < split_point], criterion, depth+1, CNT)
      self.build_tree(df_train[df_train[attr] >= split_point], df_test[df_test[attr] >= split_point], criterion, depth+1, CNT)

  def stop_criterion(self, criterion, df, depth, CNT):
    if criterion == 'max_depth':
      if depth == 5:
        return True
    if criterion == 'max_leaf':
      if CNT == 5:
        return True
    if criterion == 'purity':
      df1 = df[df['y'] == 0]
      df2 = df[df['y'] == 1]
      df3 = df[df['y'] == 2]
      if df1.shape[0] == df.shape[0] or df2.shape[0] == df.shape[0] or df3.shape[0] == df.shape[0]:
        return True
    return False
  def accuracy(self, y_test, y_pred):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    for i in range(len(y_pred)):
      if y_pred[i] == y_test[i] == 1:
        tp=tp+1
      if y_pred[i]==0 and y_pred[i]!=y_test[i]:
        fn=fn+1
      if y_pred[i]==1 and y_pred[i]!=y_test[i]:
        fp=fp+1
      if y_pred[i] == y_test[i] == 0:
        tn=tn+1
    return (tp + tn)/(tp + fp + tn + fn)

  def predict(self, df_test, criterion):
    self.build_tree(self.train, df_test, criterion, 0, -1)
    self.result.sort_index(inplace=True)
    return self.result

In [ ]:
from sklearn.model_selection import train_test_split
pd.options.mode.chained_assignment = None
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.33, random_state=6011981)

df_train=pd.DataFrame(X_train)
df_train['y']=y_train

df_test=pd.DataFrame(X_test)
df_test['y']=y_test


In [ ]:
from sklearn.metrics import accuracy_score
tree_1=DecisionTree()
tree_1.fit(df_train)
df_test_1 = tree_1.predict(df_test, 'max_leaf')
y_pred_1=df_test_1['y'].values
print("My accuracy_score: ", tree_1.accuracy(y_test, y_pred_1), " sklearn accuracy_score: ", accuracy_score(y_test, y_pred_1))
tree_2=DecisionTree()
tree_2.fit(df_train)
df_test_2 = tree_2.predict(df_test, 'max_depth')
y_pred_2=df_test_2['y'].values
print("My accuracy_score: ", tree_2.accuracy(y_test, y_pred_2), " sklearn accuracy_score: ", accuracy_score(y_test, y_pred_2))
tree_3=DecisionTree()
tree_3.fit(df_train)
df_test_3 = tree_3.predict(df_test, 'purity')
y_pred_3=df_test_3['y'].values
print("My accuracy_score: ", tree_3.accuracy(y_test, y_pred_3), " sklearn accuracy_score: ", accuracy_score(y_test, y_pred_3))

My accuracy_score:  0.9696969696969697  sklearn accuracy_score:  0.98
My accuracy_score:  0.8648648648648649  sklearn accuracy_score:  0.9
My accuracy_score:  0.8421052631578947  sklearn accuracy_score:  0.88


В итоге получилось так что взятые мной 3 критерия остановки на основе этой выборки показали высокий результат, но max_leaf лучше всего. При этом значение моей реализации функции accuracy_score сильно отличается от sklearn (sklearn работает лучше)

##  Случайный лес

Опишем алгоритм случайный лес (*random forest*) и попутно разберём основные идеи:

1. Зададим $N$ - число деревьев в лесу.
2. Для каждого $n$ из $N$ сгенерируем свою выборку $X_n$. Пусть $m$ - это количество объектов в $X$. При генерации каждой $X_n$ мы будем брать объекты $m$ раз с возвращением. То есть один и тот же объект может попасть в выборку несколько раз, а какие-то объекты не попадут. (Этот способ назвается бутстрап).
3. По каждой $X_n$ построим решающее дерево $b_n$. Обычно стараются делать глубокие деревья. В качестве критериев остановки можно использовать `max_depth` или `min_samples_leaf` (например, пока в каждом листе не окажется по одному объекту). При каждом разбиении сначала выбирается $k$ (эвристика $k = \sqrt d$, где $d$ - это число признаков объектов из выборки $X$) случайных признаков из исходных, и оптимальное разделение выборки ищется только среди них. Обратите внимание, что мы не выбрасываем оставшиеся признаки!
4. Итоговый алгоритм будет представлять собой результат голосования (для классификации) и среднее арифметическое (для регрессии). Модификация алгоритма предполагает учёт весов каждого отдельного слабого алгоритма в ансамбле, но в этом особо нет смысла.


### Задание 4.2

В качестве набора данных используйте: https://www.kaggle.com/mathchi/churn-for-bank-customers

Там есть описание и примеры работы с этими данными. Если кратко, речь идёт про задачу прогнозирования оттока клиентов. Есть данные о 10 тысячах клиентов банка, часть из которых больше не являются клиентами.

Используя либо свою реализацию, либо  `DecisionTreeClassifier` с разными настройками из `sklearn.tree` реализйте алгоритм "случайный лес". 

Найдите наилучшие гиперпараметры этого алгоритма: количество деревьев, критерий остановки, функционал качества, минимальное количество объектов в листьях и другие.

Нельзя использовать готовую реализацию случайного леса из `sklearn`.

В подобных задачах очень важна интерпретируемость алгоритма. Попытайтесь оценить информативность признаков, т.е. ответить а вопрос, значения каких признаков являются самыми важными индикаторами того, что банк потеряет клиента.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

import numpy as np
import pandas as pd
 
import random

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing

from sklearn.metrics import accuracy_score

Предобработаем данные

In [ ]:
training_data = pd.read_csv('/content/drive/MyDrive/data/churn.csv')
training_points = training_data.drop(["RowNumber", "CustomerId", "Surname", "Exited"], axis=1)
training_values = training_data.Exited
training_points.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


Применим LabelEncoder к категорильным признакам: Geography и Gender

In [ ]:
def encoding(df, features):
  X_enc = df.copy()
  training_points = pd.get_dummies(X_enc, columns=features)
  return training_points
categorial = ['Gender','Geography']
training_points = encoding(training_points, categorial)
training_points.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Gender_Female,Gender_Male,Geography_France,Geography_Germany,Geography_Spain
0,619,42,2,0.00,1,1,1,101348.88,1,0,1,0,0
1,608,41,1,83807.86,1,0,1,112542.58,1,0,0,0,1
2,502,42,8,159660.80,3,1,0,113931.57,1,0,1,0,0
3,699,39,1,0.00,2,0,0,93826.63,1,0,1,0,0
4,850,43,2,125510.82,1,1,1,79084.10,1,0,0,0,1


Разделим на тренировочную и тестовую выборки

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(training_points, training_values, test_size=0.33, random_state=42)

Реализуем алгоритм, возвращающий важность признаков

In [ ]:

class My_random_forest:
  trees = []
  columns = []
  predictions = []

  def fit(self, points, values, tree_count, k, sampsize):
    for _ in range(0, tree_count):
      points_i = points.sample(sampsize)
      values_i = values[points_i.index]
      columns_i = points_i.columns.to_numpy()
      self.columns.append(columns_i)
      tree_i = DecisionTreeClassifier(max_features=k)
      tree_params_i = {'max_depth': np.arange(1, 10),'min_samples_leaf':np.arange(1,3)}
      tree_grid_i = GridSearchCV(tree_i, tree_params_i)
      tree_grid_i.fit(points_i, values_i)
      self.trees.append(tree_grid_i)

  def predict(self, points):
    self.predictions = []
    for i in range(0, len(self.trees)):
      prediction_i = self.trees[i].predict(points[self.columns[i]])
      self.predictions.append(prediction_i)
    return self.voting()
    
  def voting(self):
    voted_predictions = []
    p = np.array(self.predictions)
    for i in range(0, p.shape[1]):
      voted_predictions.append(np.bincount(p[:,i]).argmax())
    return np.array(voted_predictions)

  def value_of_features(self, points, values):
    permutated_points = points.copy()
    feature_rating = []
    point_names = permutated_points.columns
    self.recent_accuracy = accuracy_score(values, self.predict(points))

    for i in range(0, points.shape[1]):
      permutated_points[point_names[i]] = np.array(points.iloc[:,i].sample(frac=1))
      feature_rating.append([point_names[i], accuracy_score(values, self.predict(permutated_points)) - self.recent_accuracy])
      permutated_points[point_names[i]] = np.array(points.iloc[:,i])

    return feature_rating

randfor = My_random_forest()
k_obj = int(np.sqrt(X_train.shape[1]))
view_samples_size = int(6667)
randfor.fit(X_train, y_train, 125, k_obj, view_samples_size)
pred = randfor.predict(X_test)
accuracy_score(y_test, pred)
ans = randfor.value_of_features(X_test, y_test)
print("Значимые признаки: ")
for i in range(len(ans)):
  if ans[i][1] != 0:
    print(ans[i][0], abs(ans[i][1]))

Значимые признаки: 
Age 0.05939393939393944
Tenure 0.00060606060606061
Balance 0.0060606060606061
NumOfProducts 0.038484848484848566
HasCrCard 0.00060606060606061
IsActiveMember 0.021818181818181848
EstimatedSalary 0.00121212121212122
Gender_Female 0.0015151515151515804
Gender_Male 0.00030303030303024947
Geography_France 0.00060606060606061
Geography_Spain 0.00060606060606061


Чем выше значение отклонения, тем важнее признак. Таким образом высокое значение играет кол-во продуктов, место жительства, возраст(на основе многочисленных запусков)
